# Reconhecimentos de Padrões

## Alunos: Henrique Ricardo Figueira, Higor Celante
## Dataset: sEMG for Basic Hand movements Data Set


A base de dados utilizada tem as seguintes características:

- Série Temporal
- 2 canais EMG
- 5 arquivos

Datasets:
 - 6s = 2 homens e 2 mulheres na fixa dos 22 anos executaram os 6 movimentos 30 vezes cada, cada movimento sendo coletado por 6s.
 - 5s = 1 homem executou os 6 movimentos 100 vezes cada, por 5s cada movimento.
 
Movimentos:

- Esférico
- Palma aberta
- Lateral
- Cilíndrico
- Gancho
- Pinça


![movimentos](https://i.imgur.com/0faxxKu.png)

### Importações

In [1]:
import scipy.io
import numpy as np
import os
from librosa import stft
from numpy import mean, sqrt, square, arange
from sklearn import svm

#### Carregando o Dataset

In [2]:
caminhos = [os.path.join("src", nome) for nome in os.listdir("src")]
pessoas = []

for essive in caminhos:
    if essive != "/novo.txt":
        mat = scipy.io.loadmat(essive)
        tip = [mat['tip_ch1'], mat['tip_ch2']]
        spher = [mat['spher_ch1'], mat['spher_ch2']]
        palmar = [mat['palm_ch1'], mat['palm_ch2']]
        lateral = [mat['lat_ch1'], mat['lat_ch2']]
        cilindrical = [mat['cyl_ch1'], mat['cyl_ch2']]
        hook = [mat['hook_ch1'], mat['hook_ch2']]
        data = [tip, spher, palmar, lateral, cilindrical, hook]
        data = np.array(data)
        pessoas.append(data)  
        




#print(" 6 movimentos,30 tentativas, 2 canais,  3000 coletas")


### Selecionando 1 dos Datasets

In [3]:
dataone = pessoas[2]
dataone = np.array(dataone)
dataone = np.swapaxes(dataone,1,2)

#### Shape após carregamento

> 6 movimentos, 30 tentativas, 2 canais,  3000 coletas

In [4]:
print(dataone.shape)

(6, 30, 2, 3000)


#### Segmentação

In [21]:
datax = []
segmentosize = (len(dataone[0,0,0,:]))/6
salto = segmentosize * 0.7
antpasso = segmentosize - salto


print("Sobreposicao:        ", antpasso)
print("Salto:               ",salto)
print("Tamanho do segmento: ",segmentosize)


for movimento in dataone:  
    tentativs = []
    for tentativas in movimento:
        canals = []
        for canal in tentativas:
            
            listinha = []
            listinha.append(canal[350:850])
            listinha.append(canal[700:1200])
            listinha.append(canal[1050:1550])
            listinha.append(canal[1400:1900])
            listinha.append(canal[1750:2250])
            listinha.append(canal[2100:2600])
            canals.append(np.split(canal,6) + listinha)
        tentativs.append(canals)
    datax.append( tentativs)
datax = np.array(datax)


Sobreposicao:         150.0
Salto:                350.0
Tamanho do segmento:  500.0


![segmentacao](https://i.imgur.com/rWhhCop.png)


#### Shape após segmentação

##### 6 movimentos, 30 tentativas, 2 canais,  11 segmentos de 500 valores

In [22]:
print(datax.shape)
datax = np.swapaxes(datax, 0,1)
print(datax.shape)

(6, 30, 2, 12, 500)
(30, 6, 2, 12, 500)


## Domínio do tempo

In [38]:
iemg = []
rmslist = []
varlist = []

for movimento in datax:
    listvar = []
    listiemg = []
    listrms = []
    for tentativas in movimento:

        for canal in tentativas:
            #listseg = 0.0
            segvar = []
            segrms = []
            segiemg = []


            for segmento in canal:
                
                #MAV ACUM
                mav = (abs(segmento.sum())/ 500 )
                segiemg.append(mav)
           

                #RMS ACUM
                rms = sqrt(mean(square(segmento)))
                segrms.append(rms)
                
                #VAR ACUM               
                VAR = np.var(segmento)
                segvar.append(VAR)

                
                
            #au = map(f, listseg)
            #listiemg[segiemg])

            listiemg.append(segiemg)
            listrms.append(segrms)
            listvar.append(segvar)
    iemg.append(listiemg)
    rmslist.append(listrms)
    varlist.append(listvar)

iemg = np.array(iemg)
#iemg = iemg.reshape((6,660))
rmslist = np.array(rmslist)
#rmslist = rmslist.reshape((6,660))
varlist = np.array(varlist)
#varlist = varlist.reshape((6,660))

### MAV

In [40]:

print(iemg.shape)

(30, 12, 12)


### RMS

In [34]:

print(rmslist.shape)

(30, 6, 2, 1)


### VAR

In [35]:

print(varlist.shape)

(30, 6, 2, 1)


## Domínio da frequência

#### FFT

In [356]:
from scipy.signal import welch

print(datax[:,:,:,0:11].shape)
psdf = []
fmn = []
fmd = []
cont = 0
allft = []
for lvl1 in datax:
    ffttemp = []
    fmd_temp = []
    fmn_temp = []
    som_fmd = 0
    som_fmn = 0
    for lvl2 in lvl1:
        for lvl3 in lvl2:
            #, hop_length=150
            if (cont < 11):
                temp = np.abs(stft(lvl3[cont], n_fft=500,hop_length=350))
                psd = welch(temp)
                psdf.append(psd)
                Fi = (cont * 500) / (2 * len(psd[1]))
                cont+=1
            for lvl3 in psd[1]:
                som_fmd += lvl3.sum()
                som_fmn += lvl3.sum()
                
            fmn_temp.append((Fi * som_fmn) / som_fmn)
            fmd_temp.append(0.5 * som_fmd)
            allfttemp = fmn_temp + fmd_temp
    fmd_temp = np.array(fmd_temp)
    fmd.append(fmd_temp)
    fmn_temp = np.array(fmn_temp)
    fmn.append(fmn_temp)
    allft.append(allfttemp)
    
fmd = np.array(fmd)
fmn = np.array(fmn)
psdf = np.array(psdf)
allft = np.array(allft)
print(psdf.shape)
print(fmd.shape)
print(fmn.shape)
print(allft.shape)

(6, 30, 2, 11)
(11, 2)
(6, 60)
(6, 60)
(6, 120)


#### PSD

In [357]:

print(psdf.shape)

(11, 2)


#### FMD

In [358]:

print(fmd.shape)

(6, 60)


#### FMN

In [359]:

print(fmn.shape)

(6, 60)


#### Junção do FMN e FMD, MAV, RMS, VAR

In [360]:
#allft = allft * VARLIST [:,0:120]

print(allft.shape)

(6, 120)


### Movimento 1

In [402]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.metrics import (brier_score_loss, precision_score, recall_score,f1_score)

In [425]:
teste = np.concatenate((iemg, rmslist, varlist, fmn, fmd), axis =1)
print(teste.shape)
X = teste.reshape(12600,1)
print(X.shape)

(6, 2100)
(12600, 1)


In [454]:
y = []
cont = 0


cumy = []


for i in range(6):
    y0 = []
    cont = 0
    for movimentos in teste:
        
        for sample in movimentos:
            if(cont == i):
                y0.append(1)

            
            else:
                y0.append(2)
        print(cont)
        cont+=1
        print(len(y0))
    cumy.append(y0)
print(cumy[cont-1][10500:])
    

cumy = np.array(cumy)
print(cumy.shape)


0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [455]:
#y = []
#for lvl1 in teste:
   #for lvl2 in lvl1:
    #        y.append(lvl2)
#y = np.array(y)
#print(y.shape)

In [456]:
#X = X.swapaxes(0,1)

print(X.shape)
#print(y.shape)
print(cumy[1][655:665])


(12600, 1)
[2 2 2 2 2 2 2 2 2 2]


In [458]:

ss = StandardScaler()
ss.fit(X)
ss.transform(X)
for ypsilons in cumy:
    print(ypsilons.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, ypsilons, test_size = 0.2, shuffle=True)
    #X_train, X_test, y_train, y_test = train_test_split(X, y0, test_size=0.33, random_state=42)
    #clf = OutputCodeClassifier(LinearSVC(random_state=0), code_size=2, random_state=0)
    #clf.fit(X_train, y_train).predict(X_test) 
    #y_pred = clf.fit(X_train, y_train).predict(X_test)
    #print(y_pred.shape)
    #print("\tPrecision: %1.3f" % precision_score(y_test, y_pred))


    for kernel in ['rbf']:#, 'linear']:
        for gamma in [0.001, 0.01, 0.1]:
            for C in [1, 10, 100, 1000]:
                classificador = []
                classificador = svm.SVC(gamma=gamma, C=C, kernel=kernel).fit(X_train, y_train)
                print('acuracia:', (classificador.score(X_test, y_test)) * 100, 'kernel:', kernel, 'gamma:', gamma, 'C:', C)
            

    cls = []
    cls = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto', n_components=7).fit(X_train, y_train)
    print('\nLDA acuracia:', cls.score(X_test, y_test) * 100)

(12600,)
acuracia: 83.25396825396825 kernel: rbf gamma: 0.001 C: 1
acuracia: 83.25396825396825 kernel: rbf gamma: 0.001 C: 10
acuracia: 83.25396825396825 kernel: rbf gamma: 0.001 C: 100
acuracia: 83.25396825396825 kernel: rbf gamma: 0.001 C: 1000
acuracia: 83.25396825396825 kernel: rbf gamma: 0.01 C: 1
acuracia: 83.25396825396825 kernel: rbf gamma: 0.01 C: 10
acuracia: 83.25396825396825 kernel: rbf gamma: 0.01 C: 100
acuracia: 83.25396825396825 kernel: rbf gamma: 0.01 C: 1000
acuracia: 83.25396825396825 kernel: rbf gamma: 0.1 C: 1
acuracia: 83.25396825396825 kernel: rbf gamma: 0.1 C: 10
acuracia: 83.25396825396825 kernel: rbf gamma: 0.1 C: 100
acuracia: 83.2936507936508 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 83.25396825396825
(12600,)


/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


acuracia: 84.92063492063492 kernel: rbf gamma: 0.001 C: 1
acuracia: 84.92063492063492 kernel: rbf gamma: 0.001 C: 10
acuracia: 87.97619047619048 kernel: rbf gamma: 0.001 C: 100
acuracia: 88.17460317460318 kernel: rbf gamma: 0.001 C: 1000
acuracia: 87.65873015873015 kernel: rbf gamma: 0.01 C: 1
acuracia: 87.10317460317461 kernel: rbf gamma: 0.01 C: 10
acuracia: 88.29365079365078 kernel: rbf gamma: 0.01 C: 100
acuracia: 87.97619047619048 kernel: rbf gamma: 0.01 C: 1000
acuracia: 88.65079365079364 kernel: rbf gamma: 0.1 C: 1
acuracia: 88.76984126984127 kernel: rbf gamma: 0.1 C: 10
acuracia: 88.84920634920634 kernel: rbf gamma: 0.1 C: 100
acuracia: 88.84920634920634 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 84.92063492063492
(12600,)


/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


acuracia: 83.01587301587303 kernel: rbf gamma: 0.001 C: 1
acuracia: 83.01587301587303 kernel: rbf gamma: 0.001 C: 10
acuracia: 83.01587301587303 kernel: rbf gamma: 0.001 C: 100
acuracia: 83.01587301587303 kernel: rbf gamma: 0.001 C: 1000
acuracia: 83.01587301587303 kernel: rbf gamma: 0.01 C: 1
acuracia: 83.01587301587303 kernel: rbf gamma: 0.01 C: 10
acuracia: 83.01587301587303 kernel: rbf gamma: 0.01 C: 100
acuracia: 83.01587301587303 kernel: rbf gamma: 0.01 C: 1000
acuracia: 83.01587301587303 kernel: rbf gamma: 0.1 C: 1
acuracia: 83.01587301587303 kernel: rbf gamma: 0.1 C: 10
acuracia: 83.01587301587303 kernel: rbf gamma: 0.1 C: 100
acuracia: 83.01587301587303 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 83.01587301587303
(12600,)


...

/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


acuracia: 82.57936507936508 kernel: rbf gamma: 0.001 C: 1
acuracia: 82.57936507936508 kernel: rbf gamma: 0.001 C: 10
acuracia: 82.57936507936508 kernel: rbf gamma: 0.001 C: 100
acuracia: 82.57936507936508 kernel: rbf gamma: 0.001 C: 1000
acuracia: 82.57936507936508 kernel: rbf gamma: 0.01 C: 1
acuracia: 82.57936507936508 kernel: rbf gamma: 0.01 C: 10
acuracia: 82.57936507936508 kernel: rbf gamma: 0.01 C: 100
acuracia: 82.57936507936508 kernel: rbf gamma: 0.01 C: 1000
acuracia: 82.57936507936508 kernel: rbf gamma: 0.1 C: 1
acuracia: 82.57936507936508 kernel: rbf gamma: 0.1 C: 10
acuracia: 82.57936507936508 kernel: rbf gamma: 0.1 C: 100
acuracia: 82.57936507936508 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 82.57936507936508
(12600,)


/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


acuracia: 83.80952380952381 kernel: rbf gamma: 0.001 C: 1
acuracia: 83.80952380952381 kernel: rbf gamma: 0.001 C: 10
acuracia: 83.80952380952381 kernel: rbf gamma: 0.001 C: 100
acuracia: 83.80952380952381 kernel: rbf gamma: 0.001 C: 1000
acuracia: 83.80952380952381 kernel: rbf gamma: 0.01 C: 1
acuracia: 83.80952380952381 kernel: rbf gamma: 0.01 C: 10
acuracia: 83.80952380952381 kernel: rbf gamma: 0.01 C: 100
acuracia: 83.80952380952381 kernel: rbf gamma: 0.01 C: 1000
acuracia: 83.80952380952381 kernel: rbf gamma: 0.1 C: 1
acuracia: 83.80952380952381 kernel: rbf gamma: 0.1 C: 10
acuracia: 83.80952380952381 kernel: rbf gamma: 0.1 C: 100
acuracia: 83.80952380952381 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 83.80952380952381


/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


In [ ]:


clf = SVC(gamma='auto')

X = teste
y = ["movimento1", "movimento2", "movimento3", "movimento4", "movimento5", "movimento6"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf.fit(X, y) 


In [395]:
print(clf.predict(X_test))
print(X_test)

['movimento1' 'movimento2']
[[1.68089912e-01 1.68753008e-01 1.71609190e-01 ... 7.96965344e+04
  8.09700791e+04 8.22436237e+04]
 [1.74516332e-01 1.41415272e-01 1.74720424e-01 ... 7.38655907e+04
  7.51391354e+04 7.64126801e+04]]
